## Available stocks

In [ ]:
from dotenv import load_dotenv
import pandas as pd
import requests
import datetime
import os

from sqlalchemy import text

load_dotenv()

TIINGO_API = os.environ["TIINGO_API"]

In [ ]:
headers = {
    'Content-Type': 'application/json'
}
response = requests.get(f"https://api.tiingo.com/tiingo/daily/meta?token={TIINGO_API}&columns=permaTicker,ticker,exchange,isActive", headers=headers).json()

In [ ]:
all_tickers = pd.DataFrame(response)

today = datetime.datetime.now()
all_tickers["created_at"] = today

In [ ]:
all_tickers.to_csv(f"config/supported_tickers_{today}.csv", index=False)

### Export NSYE and NASDAQ tickers

In [ ]:
import os
from src.data_ingestion.db.postgres_db import PostgresConfig, PostgresDB


db_config = PostgresConfig(
    host=os.environ["SB_DDBB_HOST"],
    port=os.environ["SB_DDBB_PORT"],
    database=os.environ["SB_DDBB_DATABASE"],
    user=os.environ["SB_DDBB_USER"],
    password=os.environ["SB_DDBB_PWD"]
)
db = PostgresDB(config=db_config)


In [ ]:
all_tickers[all_tickers["exchange"].isin(["NASDAQ", "NYSE"]) & all_tickers["isActive"]].to_sql("active_tickers", db.engine, if_exists='append', index=False)

Check database insertion

In [ ]:
# Query based
df_all_tickers = pd.read_sql(text("SELECT ticker FROM active_tickers WHERE created_at = :date_filter"), db.engine, params={"date_filter": pd.read_sql(text("SELECT max(created_at) FROM active_tickers"), db.engine)["max"].to_list()[0]})

In [ ]:
# In our object. To be decoupled
from src.data_ingestion.downloader.tiingo_ticker_downloader import TiingoDownloader

dwn = TiingoDownloader(engine=db.engine)  
dwn.available_tickers(date=None)

## End day OHLC

In [ ]:
import datetime

ticker = "aapl"
yesteday_close_date = datetime.datetime(2024, 7 ,18)

In [ ]:
from src.data_ingestion.downloader.tiingo_ticker_downloader import TiingoDownloader

dwn = TiingoDownloader(engine=db.engine)  
tickers = dwn.available_tickers(date=None)["ticker"].to_list()

In [ ]:
df = dwn.get_prices(tickers[0:5], yesteday_close_date)

In [ ]:
df

In [ ]:
df = dwn.get_daily_prices(tickers[0:5], yesteday_close_date)

In [ ]:
df

## News

In [ ]:
import os
import datetime
import pandas as pd

from src.data_ingestion.db.postgres_db import PostgresConfig, PostgresDB


db_config = PostgresConfig(
    host=os.environ["SB_DDBB_HOST"],
    port=os.environ["SB_DDBB_PORT"],
    database=os.environ["SB_DDBB_DATABASE"],
    user=os.environ["SB_DDBB_USER"],
    password=os.environ["SB_DDBB_PWD"]
)
db = PostgresDB(config=db_config)

In [ ]:
from src.data_ingestion.downloader.tiingo_ticker_downloader import TiingoDownloader

dwn = TiingoDownloader(engine=db.engine)  

In [ ]:
df_news_composed = dwn.get_news_composed(["aapl","googl"], start_date=datetime.datetime(2024, 7, 18), end_date=datetime.datetime(2024, 7, 19))

In [ ]:
df_news = dwn.get_news(["aapl","googl"], start_date=datetime.datetime(2024, 7, 18), end_date=datetime.datetime(2024, 7, 19), limit=10)

Get all the news in batch between dates

In [8]:
tickers = dwn.available_tickers(date=None)

In [ ]:
import requests
headers = {"Content-Type": "application/json"}

response = requests.get(
    f"https://api.tiingo.com/tiingo/news?token={os.environ['TIINGO_API']}&tickers=amc&startDate=2024-07-20&limit=2&offset=2",
    headers=headers,
).json()
response

In [ ]:
len(tickers['ticker'].to_list()) / 50

In [ ]:
n_elements = len(tickers['ticker'].to_list())
n_elements_per_batch = 50
batches = [(i-n_elements_per_batch,i) for i in range(n_elements_per_batch, n_elements, n_elements_per_batch)]
batches.append((batches[-1][1], n_elements))

In [ ]:
init_date = '2024-03-01'
end_date = '2024-04-01'
list_tickers = ','.join(tickers['ticker'].to_list())
ticker_batch = 50


df_news = pd.DataFrame()
for start, end in batches:
    offset = 0
    exhausted = False
    while not exhausted:
        print(offset, response)
        limit = 100
        response = requests.get(
            f"https://api.tiingo.com/tiingo/news?token={os.environ['TIINGO_API']}&tickers={list_tickers[start : end]}&startDate={init_date}&endDate={end_date}&limit={limit}&offset={offset}",
            headers=headers,
        ).json()
        offset += limit
        exhausted = len(response) == 0
        df_news = pd.concat([df_news, pd.DataFrame(response)])



In [ ]:
df_news.to_csv("news_2024_04_01_2024_05_01.csv", index=False, sep=",")

In [ ]:
dwn.get_news_composed(
    tickers=["aapl"],
    start_date=datetime.datetime(2024, 7, 22),
    end_date=datetime.datetime(2024, 7, 23),
    limit=10,
)

In [ ]:
dwn.get_daily_news_composed(
    tickers=["aapl"],
    date=datetime.datetime(2024, 7, 22),
    limit=10,
)